In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Database setup
def initialize_database():
    conn = sqlite3.connect("customer_insights.db")
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS customers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            segment TEXT,
            feedback_score INTEGER,
            campaign TEXT
        )
    ''')
    conn.commit()
    conn.close()

def insert_data(name, segment, feedback_score, campaign):
    conn = sqlite3.connect("customer_insights.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO customers (name, segment, feedback_score, campaign) VALUES (?, ?, ?, ?)",
                   (name, segment, feedback_score, campaign))
    conn.commit()
    conn.close()

def fetch_all_data():
    conn = sqlite3.connect("customer_insights.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM customers")
    data = cursor.fetchall()
    conn.close()
    return data

def fetch_data():
    conn = sqlite3.connect("customer_insights.db")
    cursor = conn.cursor()
    cursor.execute("SELECT segment, COUNT(*) FROM customers GROUP BY segment")
    data = cursor.fetchall()
    conn.close()
    return data

def create_gui():
    def add_customer():
        name = name_entry.get()
        segment = segment_combobox.get()
        feedback_score = feedback_scale.get()
        campaign = campaign_combobox.get()

        if not name or not segment or not campaign:
            messagebox.showerror("Input Error", "All fields are required.")
            return
        insert_data(name, segment, feedback_score, campaign)
        messagebox.showinfo("Success", "Customer data added successfully.")
        name_entry.delete(0, tk.END)
        segment_combobox.set("")
        feedback_scale.set(0)
        campaign_combobox.set("")
        load_data()  # Refresh table data

    def load_data():
        # Clear the treeview
        for row in tree.get_children():
            tree.delete(row)

        # Fetch data and populate the table
        data = fetch_all_data()
        for row in data:
            tree.insert("", tk.END, values=row)

    def visualize_data():
        data = fetch_data()
        segments = [row[0] for row in data]
        counts = [row[1] for row in data]
        chart_type = chart_type_combobox.get()
        if not chart_type:
            messagebox.showerror("Input Error", "Select a chart type.")
            return

        # Clear previous plots
        for widget in visualization_frame.winfo_children():
            widget.destroy()

        fig, ax = plt.subplots()
        
        if chart_type == "Bar Chart":
            ax.bar(segments, counts, color='skyblue')
            ax.set_title("Customer Segments Distribution")
            ax.set_xlabel("Segment")
            ax.set_ylabel("Count")
        elif chart_type == "Pie Chart":
            ax.pie(counts, labels=segments, autopct='%1.1f%%', startangle=140, colors=plt.cm.Paired.colors)
            ax.set_title("Customer Segments Distribution")
        elif chart_type == "Line Graph":
            ax.plot(segments, counts, marker='o', color='blue', linestyle='-', linewidth=2)
            ax.set_title("Customer Segments Trend")
            ax.set_xlabel("Segment")
            ax.set_ylabel("Count")
        else:
            messagebox.showerror("Error", "Invalid chart type.")
            return
        canvas = FigureCanvasTkAgg(fig, master=visualization_frame)
        canvas.draw()
        canvas.get_tk_widget().pack()
    
    window = tk.Tk()
    window.geometry("1000x700")
    window.title("Community Trust Bank - Customer Insights")
    
    # Input frame
    input_frame = ttk.LabelFrame(window, text="Add Customer Data")
    input_frame.pack(fill="x", padx=10, pady=5)

    ttk.Label(input_frame, text="Name:").grid(row=0, column=0, padx=5, pady=5)
    name_entry = ttk.Entry(input_frame)
    name_entry.grid(row=0, column=1, padx=5, pady=5)

    ttk.Label(input_frame, text="Segment:").grid(row=1, column=0, padx=5, pady=5)
    segment_combobox = ttk.Combobox(input_frame, values=["High-value", "Digital-first", "At-risk"], state="readonly")
    segment_combobox.grid(row=1, column=1, padx=5, pady=5)

    ttk.Label(input_frame, text="Feedback Score:").grid(row=2, column=0, padx=5, pady=5)
    feedback_scale = ttk.Scale(input_frame, from_=0, to=10, orient="horizontal")
    feedback_scale.grid(row=2, column=1, padx=5, pady=5)

    ttk.Label(input_frame, text="Campaign:").grid(row=3, column=0, padx=5, pady=5)
    campaign_combobox = ttk.Combobox(input_frame, values=["Email", "Social Media", "Other"], state="readonly")
    campaign_combobox.grid(row=3, column=1, padx=5, pady=5)

    ttk.Button(input_frame, text="Add Customer", command=add_customer).grid(row=4, columnspan=2, pady=10)

    # Data table frame
    table_frame = ttk.LabelFrame(window, text="Customer Data")
    table_frame.pack(fill="x", padx=10, pady=5)

    columns = ("ID", "Name", "Segment", "Feedback Score", "Campaign")
    tree = ttk.Treeview(table_frame, columns=columns, show="headings")
    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=100)
    tree.pack(fill="x", expand=True)

    # Visualization frame
    visualization_frame = ttk.LabelFrame(window, text="Visualize Data")
    visualization_frame.pack(fill="both", expand=True, padx=10, pady=5)

    ttk.Label(visualization_frame, text="Chart Type:").pack(pady=5)
    chart_type_combobox = ttk.Combobox(visualization_frame, values=["Bar Chart", "Pie Chart", "Line Graph"], state="readonly")
    chart_type_combobox.pack(pady=5)

    ttk.Button(visualization_frame, text="Show Insights", command=visualize_data).pack(pady=10)

    load_data()  # Load initial data into the table

    window.mainloop()

# Initialize database
initialize_database()

# Run GUI
create_gui()
